In [1]:
# !pip install monai
# !pip install nibabel

In [1]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
from monai.transforms import Spacing
from monai.transforms import SpatialPad
from monai.transforms import Resize
from monai.transforms import CropForeground

import time
import torch

In [2]:
!pwd

/workspace/jo_nb/Preprocessing


In [3]:
import sys
sys.path.insert(1, '/workspace')
from utils.utils import *

ModuleNotFoundError: No module named 'utils'

In [ ]:
os.cpu_count()

# Load Image and Label

In [14]:
raw_data_dir = '/workspace/BoneMeta_raw'

image_data_dir = '/workspace/BoneMeta_latest/images'
label_data_dir = '/workspace/BoneMeta_latest/labels'

all_1mm_image_dir = '/workspace/BoneMeta_all_1mm/images'
all_1mm_label_dir = '/workspace/BoneMeta_all_1mm/labels'

z_1mm_image_dir = '/workspace/BoneMeta_z_1mm/images'
z_1mm_label_dir = '/workspace/BoneMeta_z_1mm/labels'

data_info_path = '/workspace/DataInfo/data_info_latest.csv'


def get_label_path(file):
    return os.path.join(label_data_dir, file)

def get_image_path(file):
    return os.path.join(image_data_dir, file)

In [15]:
data_info = pd.read_csv(data_info_path, index_col=0)

In [16]:
pd.read_csv(data_info_path,index_col=0).tail()

,File Name,Lesion Size,Lesion Location,Probability,I/V,Lesion Type,pathologic fracture,annotation,Original Shape,Pixel Size,X Spacing,Y Spacing,Slice Interval,Slice Thickness,Original Center,maskPos
Case,,,,,,,,,,,,,,,,
SN110_20000101_CT Chest Lung,lesionAnnot3D-000.nii.gz,2351.041994,C7,100,V,S,NaN,C7/100/V/S,"[512,512,315]",5406,0.589844,0.589844,1.25,1.25,"[232,307,308]","(210,285,302)-(254,329,315)"
SN110_20000101_CT Chest Lung,lesionAnnot3D-001.nii.gz,12717.197555,T8,100,V,S,pathologic fracture,T8/100/V/S/pathologic fracture,"[512,512,315]",29242,0.589844,0.589844,1.25,1.25,"[261,349,186]","(230,306,176)-(292,392,196)"
SN110_20000101_CT Chest Lung,lesionAnnot3D-002.nii.gz,19585.058297,T11,100,V,M,pathologic fracture,T11/100/V/M/pathologic fracture,"[512,512,315]",45034,0.589844,0.589844,1.25,1.25,"[266,316,134]","(236,275,120)-(297,358,148)"
SN110_20000101_CT Chest Lung,lesionAnnot3D-003.nii.gz,27607.565078,L3,100,V,M,pathologic fracture,L3/100/V/M/pathologic fracture,"[512,512,315]",63481,0.589844,0.589844,1.25,1.25,"[266,240,37]","(236,201,23)-(297,279,51)"
SN110_20000101_CT Chest Lung,lesionAnnot3D-004.nii.gz,26804.749036,L4,100,V,M,pathologic fracture,L4/100/V/M/pathologic fracture,"[512,512,315]",61635,0.589844,0.589844,1.25,1.25,"[265,240,13]","(230,207,0)-(301,274,26)"


In [17]:
data_info.head()

,File Name,Lesion Size,Lesion Location,Probability,I/V,Lesion Type,pathologic fracture,annotation,Original Shape,Pixel Size,X Spacing,Y Spacing,Slice Interval,Slice Thickness,Original Center,maskPos
Case,,,,,,,,,,,,,,,,
000005_20181202_CT Abdomen+Pelvis Dynamic (contrast),lesionAnnot3D-000.nii.gz,40668.298796,L5,metastasis,V,L,pathologic fracture,L5/metastasis/V/L/pathologic fracture,"[512,512,145]",34062,0.630859,0.630859,3.0,3.0,"[298,298,53]","(242,254,43)-(355,342,63)"
000009_20180417_Chest CT (contrast),lesionAnnot3D-003.nii.gz,676.963073,T1-2,metastasis,I,S,NaN,T1-2/metastasis/I/S,"[512,512,340]",1242,0.738281,0.738281,1.0,1.0,"[285,297,301]","(277,290,293)-(293,304,309)"
000009_20180417_Chest CT (contrast),lesionAnnot3D-014.nii.gz,1834.668038,T11,metastasis,I,S,NaN,T11/metastasis/I/S,"[512,512,340]",3366,0.738281,0.738281,1.0,1.0,"[273,366,99]","(264,354,91)-(282,379,107)"
000009_20180417_Chest CT (contrast),lesionAnnot3D-018.nii.gz,613.736248,T12-4,metastasis,I,S,NaN,T12-4/metastasis/I/S,"[512,512,340]",1126,0.738281,0.738281,1.0,1.0,"[245,392,51]","(236,387,46)-(255,398,56)"
000010_20181214_CT Abdomen+Pelvis Dynamic (contrast),lesionAnnot3D-000.nii.gz,8027.330640,L4,metastasis,V,S,NaN,L4/metastasis/V/S,"[512,512,134]",6893,0.623047,0.623047,3.0,3.0,"[265,259,57]","(240,242,53)-(290,277,62)"


## Resample all 1mm

In [26]:
CT_info_path = '/workspace/DataInfo/CT_info_latest.csv'

In [29]:
CT_info = pd.read_csv(CT_info_path)

In [30]:
CT_info.head()

,Case,Original Shape,X Spacing,Y Spacing,Slice Interval,Slice Thickness,1mm Shape
0,000005_20181202_CT Abdomen+Pelvis Dynamic (con...,"[512,512,145]",0.630859,0.630859,3.0,3.0,"[323,323,435]"
1,000009_20180417_Chest CT (contrast),"[512,512,340]",0.738281,0.738281,1.0,1.0,"[378,378,340]"
2,000010_20181214_CT Abdomen+Pelvis Dynamic (con...,"[512,512,134]",0.623047,0.623047,3.0,3.0,"[319,319,402]"
3,000011_20181207_Chest CT (contrast),"[512,512,296]",0.658203,0.658203,1.0,1.0,"[337,337,296]"
4,000011_20181207_CT Abdomen+Pelvis Post (contrast),"[512,512,135]",0.679688,0.679688,3.0,3.0,"[348,348,405]"


In [32]:
SIZE_INFO_DF = CT_info.loc[:,['Case', '1mm Shape']]
SIZE_INFO_DF.drop_duplicates(inplace=True)
SIZE_INFO_DF.set_index('Case', inplace=True)
print(SIZE_INFO_DF.shape)

(471, 1)


In [33]:
SIZE_INFO_DF.loc[(SIZE_INFO_DF.index.duplicated()),:]

,1mm Shape
Case,


In [42]:
image_data_dir = '/workspace/BoneMeta_latest/images'
label_data_dir = '/workspace/BoneMeta_128/preds_boneseg/boneseg3d_fn_0_nowin_elastdf_5lyr_model_epoch100/numpy'

all_1mm_image_dir = '/workspace/BoneMeta_all_1mm/images'
all_1mm_label_dir = '/workspace/BoneMeta_all_1mm/boneseg'

os.makedirs(all_1mm_image_dir, exist_ok=True)
os.makedirs(all_1mm_label_dir, exist_ok=True)

def get_label_path(file):
    return os.path.join(label_data_dir, file)

def get_image_path(file):
    return os.path.join(image_data_dir, file)

In [43]:
def str_to_size(size_str):
    crz = size_str.strip('[]').split(',')
    torch_size = [int(value) for value in crz] 
    torch_size.reverse() # crz to zrc
    return torch_size

def get_size(file):
    case = file_to_case(file)
    size_str = SIZE_INFO_DF.loc[case]['1mm Shape']
    if not type(size_str) == str:
        size_str = size_str[0]
    tensor_size = str_to_size(size_str)
    return tensor_size

def resample_all_1mm(np_image, target_size):
    resizer = Resize(target_size)
    resized_image = resizer(np_image[np.newaxis,:]) # with channel
    return resized_image[0] # without channel

def resample_all_1mm_label(np_label, target_size):
    np_label = np_label > 0.5
    resizer = Resize(target_size)
    resized_label = resizer(np_label[np.newaxis,:]) # with channel
    return resized_label[0] # without channel

In [47]:
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0.1).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

In [37]:
from multiprocessing import Pool

def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0.5).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

def resample(file, file_dir, target_dir, resample_func, save_func):
    try:
        np_file = np.load(os.path.join(file_dir, file))
        target_size = get_size(file)
        resampled_image = resample_func(np_file, target_size)
        save_func(resampled_image, target_dir, file)
    except:
        print(file)

def wrapper(args):
    return resample(*args)

t = time.perf_counter()

file_list = os.listdir(image_data_dir)
arg_list = [(file, image_data_dir, all_1mm_image_dir, resample_all_1mm, save_image) for file in file_list]
pool = Pool(processes=32)
pool.map(wrapper, arg_list)
pool.close()
pool.join()

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')

000270_20190601_CT Abdomen+Pelvis Dynamic (contrast).npy
000424_20200312_Chest CT (noncontrast).npy
000352_20191001_CT Abdomen+Pelvis Dynamic (contrast).npy
000391_20200216_Thoracic Aorta CT Angio+3D (contrast).npy
000424_20200218_Chest CT (contrast).npy
BH069_20200505_Chest CT (contrast) + 3D.npy
000270_20190608_Chest CT (contrast).npy
BH002_20190701_Chest CT (Non contrast) + 3D.npy
000273_20190531_Chest CT (contrast).npy
BH091_20200104_Chest CT (contrast) + 3D (Chest with other CT).npy
000557_20210621_Chest CT (contrast).npy
000439_20200624_Chest CT (contrast).npy
000352_20191017_Chest CT (contrast).npy
000325_20190919_Chest CT (contrast).npy

consumed_time: 51.40529946796596s



In [48]:
t = time.perf_counter()

file_list = os.listdir(image_data_dir)
arg_list = [(file, label_data_dir, all_1mm_label_dir, resample_all_1mm_label, save_label) for file in file_list]
pool = Pool(processes=32)
pool.map(wrapper, arg_list)
pool.close()
pool.join()

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')

000424_20200312_Chest CT (noncontrast).npy
000270_20190601_CT Abdomen+Pelvis Dynamic (contrast).npy
000391_20200216_Thoracic Aorta CT Angio+3D (contrast).npy
000352_20191001_CT Abdomen+Pelvis Dynamic (contrast).npy
000424_20200218_Chest CT (contrast).npy
BH069_20200505_Chest CT (contrast) + 3D.npy
000270_20190608_Chest CT (contrast).npy
BH002_20190701_Chest CT (Non contrast) + 3D.npy
BH091_20200104_Chest CT (contrast) + 3D (Chest with other CT).npy
000273_20190531_Chest CT (contrast).npy
000557_20210621_Chest CT (contrast).npy
000439_20200624_Chest CT (contrast).npy
000325_20190919_Chest CT (contrast).npy
000352_20191017_Chest CT (contrast).npy

consumed_time: 28.506764388643205s



## Resample all same size

In [4]:
!ls /workspace

BoneMeta_192+	  Commands.md  docker	     jo_nb
BoneMeta_all_1mm  DataInfo     docker_py3.9


In [5]:
from multiprocessing import Pool

class Resampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = (np_image>0.5).astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            np_image = np_image > 0.5
        resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        np_file = np.load(os.path.join(origin_dir, file))
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=32)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

class MultiChResampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = np_image.astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            resizer = Resize(self.resample_size, mode='nearest')
        else: 
            resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        np_file = np.load(os.path.join(origin_dir, file))
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=32)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

In [7]:
!ls /workspace/BoneMeta_192+

images	   labels_V  models  small_labels_V
labels_IV  logs      preds   small_labels_VI_orig


In [8]:
source_dir = '/workspace/ExternalValidation/images'
target_dir = '/workspace/External_192+/images'

source_dir_lb = '/workspace/BoneMeta_192+/small_labels_VI_orig'
target_dir_lb = '/workspace/BoneMeta_192+/small_labels_VI'

source_dir_lb_2 = '/workspace/ExternalValidation/labels_IV'
target_dir_lb_2 = '/workspace/External_192+/labels_IV'

target_shape = [192,192,224]

image_resampler = Resampler(source_dir, target_dir, target_shape)
label_resampler_1 = Resampler(source_dir_lb, target_dir_lb, target_shape, True)
# label_resampler_2 = Resampler(source_dir_lb_2, target_dir_lb_2, target_shape, True)
# label_resampler_3 = Resampler('/workspace/BoneMeta_new/labels_sclerotic', '/workspace/BoneMeta_all_192/labels_sclerotic', [192,192,192], True)

# image_resampler.resample_all()
label_resampler_1.resample_all()
# label_resampler_2.resample_all()
# label_resampler_3.resample_all()



        consumed_time: 17.82566604297608s
        


In [6]:
source_dir = '/workspace/ExternalValidation/images'
target_dir = '/workspace/External_224+/images'

source_dir_lb = '/workspace/ExternalValidation/labels_V'
target_dir_lb = '/workspace/External_224+/labels_V'

source_dir_lb_2 = '/workspace/ExternalValidation/labels_IV'
target_dir_lb_2 = '/workspace/External_224+/labels_IV'

target_shape = [224,224,256]

image_resampler = Resampler(source_dir, target_dir, target_shape)
label_resampler_1 = Resampler(source_dir_lb, target_dir_lb, target_shape, True)
label_resampler_2 = Resampler(source_dir_lb_2, target_dir_lb_2, target_shape, True)
# label_resampler_3 = Resampler('/workspace/BoneMeta_new/labels_sclerotic', '/workspace/BoneMeta_all_192/labels_sclerotic', [192,192,192], True)

image_resampler.resample_all()
label_resampler_1.resample_all()
label_resampler_2.resample_all()
# label_resampler_3.resample_all()


        consumed_time: 4.516461744904518s
        

        consumed_time: 4.485456331633031s
        

        consumed_time: 4.566804098896682s
        


In [21]:
source_dir = '/workspace/BoneMeta_latest/images'
target_dir = '/workspace/BoneMeta_128/images'

source_dir_lb = '/workspace/BoneMeta_latest/labels_V'
target_dir_lb = '/workspace/BoneMeta_128/labels_V'

source_dir_lb_2 = '/workspace/BoneMeta_latest/labels_IV'
target_dir_lb_2 = '/workspace/BoneMeta_128/labels_IV'

target_shape = [128,128,128]

image_resampler = Resampler(source_dir, target_dir, target_shape)
label_resampler_1 = Resampler(source_dir_lb, target_dir_lb, target_shape, True)
label_resampler_2 = Resampler(source_dir_lb_2, target_dir_lb_2, target_shape, True)
# label_resampler_3 = Resampler('/workspace/BoneMeta_new/labels_sclerotic', '/workspace/BoneMeta_all_192/labels_sclerotic', [192,192,192], True)

image_resampler.resample_all()
label_resampler_1.resample_all()
label_resampler_2.resample_all()
# label_resampler_3.resample_all()


        consumed_time: 159.7472921013832s
        

        consumed_time: 79.46646624896675s
        

        consumed_time: 79.34824098274112s
        


In [95]:
np.load('/workspace/BoneMeta_all_128/images/BH008_20190701_abdomen.npy').shape

(128, 128, 128)

In [10]:
file_names = os.listdir(all_128_image_dir)
image_sizes = [np.load(os.path.join(all_128_image_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(all_128_label_dir, file)).shape for file in file_names]

In [11]:
label_sizes

[(128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128

# Test Codes

## Testing One Case

In [64]:
img = np.load(get_image_path('000251_20190703_chest.npy'))

In [73]:
cropper = CropForeground()

In [78]:
cropper.compute_bounding_box(img[np.newaxis,:])

(array([ 0, 58,  0]), array([297, 512, 512]))

In [79]:
cropper(img).shape

(297, 454, 512)

In [89]:
import time

In [90]:
t = time.perf_counter()

resampled_image = resample_all_1mm(img, ct_info.resolution[0])

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 1.6059333749872167s



In [53]:
img.shape

(297, 512, 512)

In [54]:
resampled_image.shape

(297, 287, 287)

In [56]:
label = np.load(get_label_path('000251_20190703_chest.npy'))

In [57]:
resampled_label = resample_all_1mm(label, ct_info.resolution[0])

In [77]:
adjust_window(resampled_image, [700,100]).astype(np.int32).min()

0

In [ ]:
plot_image_and_label(adjust_window(resampled_image, [700,100]).astype(np.int32), (resampled_label>0).astype(np.uint8))

In [29]:
from monai.transforms import Rand3DElastic
sample_np_image = dcm_to_numpy(cases[0], dcm_base_dir) # npy 로 변환한 3D 이미지 하나 입니다
sample_label = np.load('/workspace/BoneMeta_new/labels/{}.npy'.format(cases[0]))
elastic_deform = Rand3DElastic(sigma_range=(35,35), magnitude_range=(10,10), prob=1.0, rotate_range=0, shear_range=0)
deformed_image = elastic_deform(sample_np_image)


## Testing All Sizes

In [114]:
file_names = os.listdir(all_1mm_image_dir)

In [115]:
image_sizes = [np.load(os.path.join(all_1mm_image_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(all_1mm_label_dir, file)).shape for file in file_names]

In [116]:
image_sizes == label_sizes

True

In [ ]:
plot_image_and_label(np.load(os.path.join(all_1mm_image_dir, file_names[5])), np.load(os.path.join(all_1mm_label_dir, file_names[5])))